In [4]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357273 sha256=f318b749612c48f4989d7f3c85914f6667ac8be1ab4854254b9a528d67eb02b0
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [5]:
# Import libraries
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate

# Upload datasets
movies_file = "/content/drive/MyDrive/movies.csv"
ratings_file = "/content/drive/MyDrive/ratings_cleaned.csv"
# Load datasets
movies = pd.read_csv(movies_file)
ratings = pd.read_csv(ratings_file)

In [6]:
print("Movies Dataset:")
print(movies.head())
print("\nRatings Dataset:")
print(ratings.head())

Movies Dataset:
   movieId                         title  \
0    42594       Scream and Scream Again   
1   280133                         Jenny   
2   225155      The Adventures of Gerard   
3    85255      ...tick...tick...tick...   
4   117999  Last of the Mobile Hot Shots   

                                       cast_and_crew  year  \
0  Gordon Hessler (director); Christopher Wicking...  1970   
1  George Bloomfield (director/screenplay); Marti...  1970   
2  Jerzy Skolimowski (director/screenplay); Arthu...  1970   
3  Ralph Nelson (director); James Lee Barrett (sc...  1970   
4  Sidney Lumet (director); Gore Vidal (screenpla...  1970   

                        poster_path                     genre  
0  /49m9QJ2ubKuVtdDj9B7XqgCyriv.jpg   Horror, Science Fiction  
1  /iaJ0PyRh17KoMcUlXxTVu6Vi85s.jpg            Drama, Romance  
2  /aoOV7vWvGiNjaUTHCSIEAeHSvOw.jpg  Adventure, Comedy, Drama  
3  /rBNgytHXZEXgOl805pYWKiBQe7s.jpg             Drama, Action  
4  /xj5Rub6H5B9yMh6GL3oomO

In [14]:
movies.drop(columns=['poster_path'], inplace=True)

In [15]:
# Ensure the required columns are present
assert {'userId', 'movieId', 'rating'}.issubset(ratings.columns), "Ratings file must contain 'userId', 'movieId', 'rating'."
assert {'movieId', 'title'}.issubset(movies.columns), "Movies file must contain 'movieId' and 'title'."
# Define the Reader for Surprise (rating scale should match the dataset)
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))

# Load the data into Surprise's Dataset format
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# Train-test split
trainset, testset = train_test_split(data, test_size=0.2)

# Train the SVD (Singular Value Decomposition) model
model = SVD()
model.fit(trainset)


In [16]:
# Evaluate the model
print("\nModel Evaluation Results:")
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Model Evaluation Results:
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.0799  2.0229  2.0416  2.0457  2.0706  2.0522  0.0206  
MAE (testset)     1.5955  1.5616  1.5768  1.5585  1.5846  1.5754  0.0139  
Fit time          0.25    0.23    0.22    0.23    0.23    0.23    0.01    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([2.07994405, 2.02287764, 2.04163993, 2.04572788, 2.07062193]),
 'test_mae': array([1.59546081, 1.56161031, 1.57677305, 1.55851304, 1.58462577]),
 'fit_time': (0.25319766998291016,
  0.22903895378112793,
  0.2235558032989502,
  0.22930431365966797,
  0.22967958450317383),
 'test_time': (0.010662317276000977,
  0.010432243347167969,
  0.010432004928588867,
  0.010371923446655273,
  0.01098179817199707)}

In [17]:
# Function to get movie recommendations for a user
def get_recommendations(user_id, model, movies, ratings, top_n=10):
    """
    Generate top N movie recommendations for a given user.
    """
    # Get all movies the user has already rated
    user_rated_movies = ratings[ratings['userId'] == user_id]['movieId']

    # Find unrated movies
    unrated_movies = movies[~movies['movieId'].isin(user_rated_movies)]

    # Predict ratings for all unrated movies
    recommendations = []
    for movie_id in unrated_movies['movieId']:
        prediction = model.predict(user_id, movie_id)
        recommendations.append((movie_id, prediction.est))

    # Sort by estimated rating in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Get top N recommendations
    top_recommendations = recommendations[:top_n]

    # Display recommended movies with titles
    print(f"\nTop {top_n} Movie Recommendations for User {user_id}:")
    for movie_id, predicted_rating in top_recommendations:
        movie_title = movies[movies['movieId'] == movie_id]['title'].values[0]
        print(f"{movie_title} (Predicted Rating: {predicted_rating:.2f})")

In [18]:
# Get recommendations for a specific user
user_id = 2  # Replace this with the desired userId
get_recommendations(user_id, model, movies, ratings, top_n=10)


Top 10 Movie Recommendations for User 2:
Top Gun: Maverick (Predicted Rating: 8.02)
Interstellar (Predicted Rating: 7.95)
Joker (Predicted Rating: 7.94)
Spider-Man: No Way Home (Predicted Rating: 7.90)
The Lighthouse (Predicted Rating: 7.73)
Blade Runner (Predicted Rating: 7.68)
Terminator 2: Judgment Day (Predicted Rating: 7.67)
Guardians of the Galaxy (Predicted Rating: 7.64)
Die Hard (Predicted Rating: 7.63)
The Empire Strikes Back (Predicted Rating: 7.63)
